# Intro to PyTorch

After using TensorFlow and Scikit-Learn, I am finding all the frameworks probably have a similar feel.   The key things to learn:

- How the framework wants its arrays -- numpy, custom, pandas ?
- How to use the frameworks wrappers (if there are any)
- Row or column orientation ?   Training examples stacked rows or columns
- One-hot encoded Y-expected results ?

PyTorch has its own set of wrappers, so you have to convert from ndarray<->torch structures.  Everything in Torch should be in Tensor structures.


In [8]:
import torch
import numpy

torchTensorA = torch.from_numpy(numpy.array([[1,2,3],[4,5,6]]))
torchTensorB = torch.from_numpy(numpy.array([8,8,8]))

ttC1 = torchTensorA + torchTensorB
ttC2 = torch.add(torchTensorA,torchTensorB)

print (ttC1, '\n', ttC2, '\n', ttC1==ttC2)

np1 = ttC1.numpy()
print (np1, type(np1))

tensor([[  9,  10,  11],
        [ 12,  13,  14]], dtype=torch.int32) 
 tensor([[  9,  10,  11],
        [ 12,  13,  14]], dtype=torch.int32) 
 tensor([[ 1,  1,  1],
        [ 1,  1,  1]], dtype=torch.uint8)
[[ 9 10 11]
 [12 13 14]] <class 'numpy.ndarray'>


In [18]:
# Torch has CUDA GPU support.
print ('cuda?', torch.cuda.is_available())

# Using it isn't too hard but TF seems easier:  https://pytorch.org/docs/stable/notes/cuda.html
cuda = torch.device("cuda:0") 
if (torch.cuda.is_available()):
    x = torch.empty((8, 42), device=cuda)  # if u have a GPU


cuda? False


## Basic Logistic Regression

Yet another Logistic Regression Example !


In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch, os, sys
import numpy as np
from sklearn.utils import shuffle
from myutils import *
    
# boilerplate setup my data
data, yarr, features, fnames = getGagaData(maxrows=200, maxfeatures=2000, gtype=None, stopwords='english')
xMatrix = shuffle(data, random_state=0)
yArr = shuffle(np.array(yarr).reshape(-1,1), random_state=0)
partition = int(.70*len(yArr))
trainingX = xMatrix[:partition]
trainingY = yArr[:partition]
testX = xMatrix[partition:]
testY = yArr[partition:]

# Create random input and output data
dtype = torch.float
x = torch.tensor(trainingX, dtype=dtype)
y = torch.tensor(trainingY, dtype=dtype)  
testx = torch.tensor(testX, dtype=dtype)
testy = torch.tensor(testY, dtype=dtype)  

# Randomly initialize weights
torch.manual_seed(0)
w1 = torch.randn(len(features),1, dtype=dtype)

# gradient descent
learning_rate = 0.1
for t in range(25000):
    h = x.mm(w1)               # each feature * weight
    y_pred = h.sigmoid()

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()  # item?
    if (t % 5000 == 0):
        log.warn('loop %d, %.8f'%(t, loss))

    grad_w1 = x.t().mm(y_pred - y)

    # Update weights using gradient descent
    w1 -= learning_rate * grad_w1

print('training complete ', loss)
print('test validation phase')

h = testx.mm(w1)               # matrixMult or dot prod == same?
y_pred = h.sigmoid().round()

# y_pred = h.mm(w2)
# y_pred_sig = y_pred.sigmoid()
log.debug('ytest', pandas.DataFrame(testy.numpy()).head())
log.debug('ypred', pandas.DataFrame(y_pred.numpy()).head())

# Compute and print loss after rounding to 0/1's
testDiffs = (y_pred - testy)
p = pandas.DataFrame(testDiffs.numpy())
log.debug('diffs', p.head())
tests = len(p)
correct = len(p[(p[0] == 0)])
print('total correct/tests', correct, tests)
print('correct % =', round((correct/tests)*100, 2))


I won't bore everyone with the details above since its documented in prior notebooks.

Onto the next step <B>Torch Neural Networks</B>, doing it the manual way first:
